In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# tf.contrib.layers.fully_connected
```python
tf.contrib.layers.fully_connected(
    inputs,
    num_outputs,
    activation_fn=tf.nn.relu,
    normalizer_fn=None,
    normalizer_params=None,
    weights_initializer=initializers.xavier_initializer(),
    weights_regularizer=None,
    biases_initializer=tf.zeros_initializer(),
    biases_regularizer=None,
    reuse=None,
    variables_collections=None,
    outputs_collections=None,
    trainable=True,
    scope=None
)
```

Note that if inputs have a rank greater than 2, then inputs is flattened prior to the initial matrix multiply by weights.

**Args**

- inputs: 秩至少为 2 的张量，且其最后一维应为静态值，如`[batch_size, depth]`或`[None, None, None, channels]`；注意，如果输入的秩大于 2，那么输入会在与权重相乘之前扁平化；秩小于 2 或其最后一维没有设定时会报错

- num_outputs: 输出的神经元个数

- activation_fn: 默认 ReLU ，可设为None

- normalizer_fn: 用于代替`biases`的归一化函数；若指明了`normalizer_fn`，则忽略 `biases_initializer`和`biases_regularizer`，默认为 None

- normalizer_params: 归一化函数的参数

- weights_initializer: 权重的 initializer，默认为`tf.contrib.layers.xavier_initializer`

- weights_regularizer: 权重的 regularizer

- biases_initializer: 偏置的 initializer，None 则不添加偏置

- biases_regularizer: 偏置的 regularizer

- reuse: 该隐藏层及其变量是否可以重用；若要重用，则必须给出作用域

- variables_collections: 所有变量的集合组成的列表，或所包含的变量对应集合列表互不相同的字典

- outputs_collections: 添加输出的集合

- trainable: True 则将所有变量添加至计算图集合`GraphKeys.TRAINABLE_VARIABLES`中

- scope: variable_scope.

In [ ]:
tf.contrib.layers.fully_connected()

#  

#  

# tf.contrib.layers.batch_norm
```python
tf.contrib.layers.batch_norm(
    inputs,
    decay=0.999,
    center=True,
    scale=False,
    epsilon=0.001,
    activation_fn=None,
    param_initializers=None,
    param_regularizers=None,
    updates_collections=tf.GraphKeys.UPDATE_OPS,
    is_training=True,
    reuse=None,
    variables_collections=None,
    outputs_collections=None,
    trainable=True,
    batch_weights=None,
    fused=None,
    data_format=DATA_FORMAT_NHWC,
    zero_debias_moving_mean=False,
    scope=None,
    renorm=False,
    renorm_clipping=None,
    renorm_decay=0.99,
    adjustment=None
)
```

原论文[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](http://arxiv.org/abs/1502.03167.)

可以用作`conv2d`和`fully_connected`的`normalizer`函数；如果`data_format`是`NHWC`，则对最后一个维度之外的所有维度进行标准化；如果`data_format`是`NCHW`，则对最后第二维度之外的所有维度进行标准化；对于 2D 张量，该操作在 batch 的维度上进行，对于 4D 张量，该操作在 batch 和长、宽维度上进行


Note: when training, the moving_mean and moving_variance need to be updated. By default the update ops are placed in tf.GraphKeys.UPDATE_OPS, so they need to be added as a dependency to the train_op. For example:

```python
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss)
```
  
One can set `updates_collections=None` to force the updates in place, but that can have a speed penalty, especially in distributed settings.

**Args**

- inputs: 一个 2D 或更高维的张量，其中第一个维数是`batch_size` The normalization is over all but the last dimension if data_format is NHWC and the second dimension if data_format is NCHW.

- decay: moving mean 的衰减常数，`decay`合理值接近1.0，如0.999、0.99、0.9，等等。如果模型的训练表现相当好，但验证集和/或测试集表现不佳时可以使用更低一些的衰减值(建议尝试`decay=0.9`)；若想提高稳定性，可以尝试`zero_debias_moving_mean=True`

- center: 如果为真，则在标准化张量上加上的偏移量。如果为假，则忽略`beta`

- scale: True 时乘以 gamma，否则不使用gamma；当下一层是线性函数时，`nn.relu`，可以不使用这个，因为缩放可以由下一层完成

- epsilon: 方差所加的小量，避免除以零

- activation_fn: 激活函数，默认 None

- param_initializers: beta、gamma、moving mean、moving variance 的初始化函数

- param_regularizers: beta、gamma 的正则化函数

- updates_collections: Collections to collect the update ops for computation. The updates_ops need to be executed with the train_op. If None, a control dependency would be added to make sure the updates are computed in place.

- is_training: 该层是否处于训练模式；在训练模式中，it would accumulate the statistics of the moments into `moving_mean` and `moving_variance` using an exponential moving average with the given `decay`；非训练模式则会使用`moving_mean`和`moving_variance`的值

- reuse: 该层及其变量是否可以重用；若要重用，则必须给出作用域

- variables_collections: 变量的集合

- outputs_collections: 输出的集合

- trainable: True 时将变量添加到计算图集合`GraphKeys.TRAINABLE_VARIABLES`

- batch_weights: 形状为`[batch_size]`的张量，即每个 batch 的频率权重 (frequency weight)，若指明了，则批归一化使用加权平均值和方差，这可以用来修正训练示例选择中的 bias 偏差

- fused: None 或 True 时，可能的话则使用更快的融合方式去实现批归一化，False 时使用系统推荐方式实现

- data_format: 字符串，默认`NHWC`，也可以是`NCHW`，其他形式会抛出`ValueError`

- zero_debias_moving_mean: 布尔型，是否为`moving_mean`使用`zero_debias`，这会产生一对新变量`moving_mean/biased`和`moving_mean/local_step`

- scope: `variable_scope`

- renorm: 是否使用[Batch Renormalization](https://arxiv.org/abs/1702.03275)，会在训练时增加额外变量 The inference is the same for either value of this parameter

- renorm_clipping: 一个字典，可以将键`rmax`，`rmin`，`dmax`映射到用于 clip renorm correction 的标量张量；correction`(r, d)`会以`corrected_value = normalized_value * r + d`的形式表现，其中`r`的 clip 为[rmin, rmax]，`d`的 clip 为[-dmax, dmax]；未指明的`rmax`、`rmin`、`dmax`会分别设置为`inf`、`0`、`inf`

- renorm_decay: Momentum used to update the moving means and standard deviations with renorm. Unlike momentum, this affects training and should be neither too small (which would add noise) nor too large (which would give stale estimates). Note that decay is still applied to get the means and variances for inference.

- adjustment: A function taking the Tensor containing the (dynamic) shape of the input tensor and returning a pair (scale, bias) to apply to the normalized values (before gamma and beta), only during training. For example, adjustment = lambda shape: ( tf.random_uniform(shape[-1:], 0.93, 1.07), tf.random_uniform(shape[-1:], -0.1, 0.1)) will scale the normalized value by up to 7% up or down, then shift the result by up to 0.1 (with independent scaling and bias for each feature but shared across all examples), and finally apply gamma and/or beta. If None, no adjustment is applied.


#  

#  

# tf.contrib.layers.xavier_initializer()
```python
tf.contrib.layers.xavier_initializer(
    uniform=True,
    seed=None,
    dtype=tf.float32
)
```

返回对权重执行“Xavier”初始化的初始化函数，原论文[Understanding the difficulty of training deep feedforward neural networks. International conference on artificial intelligence and statistics](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

该初始化函数用来保持所有层中梯度的放缩比例大致相同；均匀分布中，这个范围是`x = sqrt(6. / (in + out)); [-x, x]`；正态分布中则使用`sqrt(2 / (in + out))`的标准差

**Args**

- uniform: 使用均匀分布还是正态分布随机初始化

- seed: 随机种子，见`tf.set_random_seed`

- dtype: 只支持浮点型数据类型